# Preparing Chicago Census Data at Zip Code Level

In [485]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import censusdata
import requests
import json
from tqdm import tqdm
pd.set_option('display.max_columns', 100)

## Preparing ACS5 dataset for use in analysis
In this dataset, I'll be extracting the following features for use in the analysis:
- Mean HH size
- Median Age
- Percent Hispanic
- Percent Black
- Percent White
- Percent Undocumented
- Percent Uninsured
- Percent Unemployed
- Percent Health Workers

In [486]:
# import list of variables necessary for analysis
variable_list = list(pd.read_excel('acs_variables.xlsx')['raw_name'].values)
variable_list = [each+'E' for each in variable_list]
variables = ','.join(variable_list)

In [504]:
# Requests Census data through Census API
request_string = 'https://api.census.gov/data/2018/acs/acs5?get={}&for=zip%20code%20tabulation%20area:*'.format(variables)
data = requests.get(request_string)


In [505]:
# Convert the data to a dataframe
df = pd.DataFrame(data.json()[1:], columns = data.json()[0])

In [506]:
df.head()

,B01003_001E,B01001_002E,B01001_026E,B08201_001E,B01002_001E,B02001_002E,B02001_003E,B03001_003E,B03001_004E,B05001_006E,B05002_026E,B08014_002E,B27010_002E,B27010_017E,B27010_018E,B27010_033E,B27010_034E,B27010_050E,B27010_051E,B27010_066E,B19013_001E,C24030_023E,C24030_050E,B23025_001E,B23025_002E,B23025_003E,B23025_004E,B08101_001E,B08101_009E,B08101_017E,B08101_025E,B08101_033E,B08101_041E,B08101_049E,zip code tabulation area
0,17242,8426,8816,5517,40.5,13026,145,17184,0,None,59,159,3991,139,3540,314,6666,513,2989,42,13092,108,381,13924,5811,5811,3495,3453,3024,170,0,145,44,70,00601
1,38442,18842,19600,12738,42.3,30529,1070,35925,83,None,214,595,7827,270,7920,869,16049,1031,6600,79,16358,347,777,32020,13493,13463,11536,11444,7917,2448,32,407,165,475,00602
2,48814,23939,24875,19233,41.1,37330,1930,47551,68,None,205,456,10790,312,9000,827,17967,1249,9608,128,16603,306,839,40180,15595,15524,12400,11908,10524,612,35,402,285,50,00603
3,6437,3212,3225,2014,43.3,2627,149,6427,0,None,162,54,1343,41,1229,32,2707,101,1158,23,12832,34,126,5365,1585,1585,1355,1355,1153,8,0,170,4,20,00606
4,27073,13112,13961,8858,42.1,20451,696,26043,56,None,39,493,5666,0,5331,414,10834,604,5157,5,19309,155,541,22641,9754,9754,8464,8391,5882,1827,23,176,219,264,00610


In [507]:
# Import variable names crosswalk
variable_names = pd.read_excel('acs_variables.xlsx')[['name', 'raw_name']]

In [508]:
variable_names.raw_name = variable_names.raw_name.apply(lambda x: x+'E')

In [509]:
crosswalk = variable_names[variable_names.columns[::-1]]

In [510]:
crosswalk.set_index('raw_name', inplace=True)

In [511]:
new_names = []
for each in list(df.columns[:-1]):
    new_names.append(crosswalk.loc[each][0])  

In [512]:
new_names.append('ZCTA')

In [513]:
df.columns = new_names

In [514]:
df.head()

,totalPop,male,female,totalHH,medianAge,race.white,race.black,hispanic,hispanic.mex,nonCitizen,nonCitizenFBLA,noVehiclesAvail,total__18,unins__18,total_19_34,unins_19_34,total_35_64,unins_35_64,total_65_,unins_65_,medianHHInc,healthWorkers.f,healthWorkers.m,employment.pop,employment.lf,employment.clf,employment.emp,commute.tot,commute.drove,commute.carpooled,commute.transit,commute.walked,commute.other,commute.workAtHome,ZCTA
0,17242,8426,8816,5517,40.5,13026,145,17184,0,None,59,159,3991,139,3540,314,6666,513,2989,42,13092,108,381,13924,5811,5811,3495,3453,3024,170,0,145,44,70,00601
1,38442,18842,19600,12738,42.3,30529,1070,35925,83,None,214,595,7827,270,7920,869,16049,1031,6600,79,16358,347,777,32020,13493,13463,11536,11444,7917,2448,32,407,165,475,00602
2,48814,23939,24875,19233,41.1,37330,1930,47551,68,None,205,456,10790,312,9000,827,17967,1249,9608,128,16603,306,839,40180,15595,15524,12400,11908,10524,612,35,402,285,50,00603
3,6437,3212,3225,2014,43.3,2627,149,6427,0,None,162,54,1343,41,1229,32,2707,101,1158,23,12832,34,126,5365,1585,1585,1355,1355,1153,8,0,170,4,20,00606
4,27073,13112,13961,8858,42.1,20451,696,26043,56,None,39,493,5666,0,5331,414,10834,604,5157,5,19309,155,541,22641,9754,9754,8464,8391,5882,1827,23,176,219,264,00610


In [550]:
# Restrict df to Chicago zip codes only
chicago_zips = df[(df['ZCTA']>='60601')&(df['ZCTA']<='60701')|(df['ZCTA']=='60707')|(df['ZCTA']=='60827')]

In [525]:
chicago_zips.shape

(58, 35)

In [552]:
chicago_zips = chicago_zips.astype(float)
chicago_zips.ZCTA = chicago_zips.ZCTA.astype(int)

In [553]:
# Create features of interest
chicago_zips['mean_HHsize'] = chicago_zips['totalPop']/chicago_zips['totalHH']
chicago_zips['pctHispanic'] = chicago_zips['hispanic']/chicago_zips['totalPop']
chicago_zips['pctBlack'] = chicago_zips['race.black']/chicago_zips['totalPop']
chicago_zips['pctWhite'] = chicago_zips['race.white']/chicago_zips['totalPop']
chicago_zips['pctUndocumented'] = chicago_zips['nonCitizen']/chicago_zips['totalPop']
chicago_zips['pctUndocumentedFBLA'] = chicago_zips['nonCitizenFBLA']/chicago_zips['totalPop']
chicago_zips['pctUninsured'] = (chicago_zips['unins__18']+chicago_zips['unins_19_34']+chicago_zips['unins_35_64']+
                                chicago_zips['unins_65_'])/(chicago_zips['total__18']+chicago_zips['total_19_34']+
                                                          chicago_zips['total_35_64']+chicago_zips['total_65_'])
chicago_zips['pctUnemployed'] = (chicago_zips['employment.clf']-chicago_zips['employment.emp'])/chicago_zips['totalPop']
chicago_zips['pctHealthWorkers'] = (chicago_zips['healthWorkers.f'] + chicago_zips['healthWorkers.m'])/chicago_zips['totalPop']

In [554]:
chicago_zips.head()

,totalPop,male,female,totalHH,medianAge,race.white,race.black,hispanic,hispanic.mex,nonCitizen,nonCitizenFBLA,noVehiclesAvail,total__18,unins__18,total_19_34,unins_19_34,total_35_64,unins_35_64,total_65_,unins_65_,medianHHInc,healthWorkers.f,healthWorkers.m,employment.pop,employment.lf,employment.clf,employment.emp,commute.tot,commute.drove,commute.carpooled,commute.transit,commute.walked,commute.other,commute.workAtHome,ZCTA,mean_HHsize,pctHispanic,pctBlack,pctWhite,pctUndocumented,pctUndocumentedFBLA,pctUninsured,pctUnemployed,pctHealthWorkers
20694,14675.0,7191.0,7484.0,8864.0,34.9,10884.0,818.0,1274.0,468.0,3513.0,520.0,3761.0,1191.0,18.0,6166.0,310.0,5215.0,111.0,2075.0,0.0,103243.0,499.0,727.0,13969.0,9828.0,9804.0,9212.0,9191.0,2136.0,267.0,1638.0,3977.0,461.0,712.0,60601,1.655573,0.086814,0.055741,0.741670,0.239387,0.035434,0.029972,0.040341,0.083543
20695,1244.0,693.0,551.0,602.0,30.6,848.0,47.0,81.0,70.0,146.0,59.0,347.0,161.0,3.0,679.0,8.0,399.0,0.0,5.0,0.0,157125.0,26.0,131.0,1096.0,931.0,931.0,930.0,930.0,80.0,0.0,233.0,362.0,20.0,235.0,60602,2.066445,0.065113,0.037781,0.681672,0.117363,0.047428,0.008842,0.000804,0.126206
20696,1174.0,573.0,601.0,564.0,28.9,745.0,38.0,115.0,13.0,188.0,0.0,264.0,140.0,23.0,548.0,59.0,374.0,0.0,112.0,0.0,151765.0,73.0,21.0,1118.0,819.0,819.0,754.0,754.0,141.0,44.0,115.0,410.0,0.0,44.0,60603,2.081560,0.097956,0.032368,0.634583,0.160136,0.000000,0.069847,0.055366,0.080068
20697,782.0,369.0,413.0,479.0,32.4,496.0,44.0,34.0,6.0,127.0,7.0,206.0,54.0,0.0,381.0,0.0,254.0,3.0,93.0,0.0,116250.0,14.0,39.0,751.0,547.0,547.0,502.0,498.0,58.0,17.0,77.0,274.0,18.0,54.0,60604,1.632568,0.043478,0.056266,0.634271,0.162404,0.008951,0.003836,0.057545,0.067775
20698,27519.0,13138.0,14381.0,13865.0,33.9,16843.0,4727.0,1608.0,737.0,2851.0,377.0,4198.0,3157.0,132.0,10951.0,361.0,10171.0,216.0,2569.0,0.0,111164.0,861.0,1941.0,25321.0,18837.0,18767.0,18075.0,17689.0,5489.0,643.0,5075.0,3951.0,1328.0,1203.0,60605,1.984782,0.058432,0.171772,0.612050,0.103601,0.013700,0.026408,0.025146,0.101821


In [555]:
chicago_zips.columns

Index(['totalPop', 'male', 'female', 'totalHH', 'medianAge', 'race.white',
       'race.black', 'hispanic', 'hispanic.mex', 'nonCitizen',
       'nonCitizenFBLA', 'noVehiclesAvail', 'total__18', 'unins__18',
       'total_19_34', 'unins_19_34', 'total_35_64', 'unins_35_64', 'total_65_',
       'unins_65_', 'medianHHInc', 'healthWorkers.f', 'healthWorkers.m',
       'employment.pop', 'employment.lf', 'employment.clf', 'employment.emp',
       'commute.tot', 'commute.drove', 'commute.carpooled', 'commute.transit',
       'commute.walked', 'commute.other', 'commute.workAtHome', 'ZCTA',
       'mean_HHsize', 'pctHispanic', 'pctBlack', 'pctWhite', 'pctUndocumented',
       'pctUndocumentedFBLA', 'pctUninsured', 'pctUnemployed',
       'pctHealthWorkers'],
      dtype='object')

In [556]:
census_df = chicago_zips[['ZCTA', 'totalPop', 'medianAge', 'medianHHInc', 'mean_HHsize', 'pctHispanic', 'pctBlack',
             'pctWhite', 'pctUndocumented', 'pctUndocumentedFBLA', 'pctUninsured', 'pctUnemployed',
             'pctHealthWorkers']]

In [558]:
census_df.shape

(58, 13)

## Gathering Essential Worker Census Data
The code below will be used to determine the percentage of essential workers in a given area. Essential workers are those who, during the start of the spread of COVID-19 in Chicago, were able to continue working due to the role of their work in preserving everyday necessities.

In [559]:
# Import list of variables necessary for analysis
worker_list = list(pd.read_excel('acs_occupation_lookup.xlsx').variable)[:-1]
worker_list = [each+'E' for each in worker_list]
worker_list = ','.join(worker_list)

In [560]:
# Requests occupation data through Census API from 
request_string2 = 'https://api.census.gov/data/2018/acs/acs5/subject?get={},group(S2401)&for=zip%20code%20tabulation%20area:*'.format(worker_list)
data2 = requests.get(request_string2)


In [561]:
df2 = pd.DataFrame(data2.json()[1:], columns = data2.json()[0])

In [562]:
df2.head()

,S2401_C01_001E,S2401_C01_002E,S2401_C01_003E,S2401_C01_004E,S2401_C01_005E,S2401_C01_006E,S2401_C01_007E,S2401_C01_008E,S2401_C01_009E,S2401_C01_010E,S2401_C01_011E,S2401_C01_012E,S2401_C01_013E,S2401_C01_014E,S2401_C01_015E,S2401_C01_016E,S2401_C01_017E,S2401_C01_018E,S2401_C01_019E,S2401_C01_020E,S2401_C01_021E,S2401_C01_022E,S2401_C01_023E,S2401_C01_024E,S2401_C01_025E,S2401_C01_026E,S2401_C01_027E,S2401_C01_028E,S2401_C01_029E,S2401_C01_030E,S2401_C01_031E,S2401_C01_032E,S2401_C01_033E,S2401_C01_034E,S2401_C01_035E,S2401_C01_036E,GEO_ID,S2401_C01_001E,S2401_C01_001M,S2401_C02_001E,S2401_C02_001M,S2401_C03_001E,S2401_C03_001M,S2401_C04_001E,S2401_C04_001M,S2401_C05_001E,S2401_C05_001M,S2401_C01_002E,S2401_C01_002M,S2401_C02_002E,...,S2401_C01_032MA,S2401_C02_032MA,S2401_C02_032EA,S2401_C03_032EA,S2401_C03_032MA,S2401_C04_032MA,S2401_C04_032EA,S2401_C05_032MA,S2401_C05_032EA,S2401_C01_033MA,S2401_C01_033EA,S2401_C02_033EA,S2401_C02_033MA,S2401_C03_033EA,S2401_C03_033MA,S2401_C04_033MA,S2401_C04_033EA,S2401_C05_033MA,S2401_C05_033EA,S2401_C01_034EA,S2401_C01_034MA,S2401_C02_034MA,S2401_C02_034EA,S2401_C03_034MA,S2401_C03_034EA,S2401_C04_034EA,S2401_C04_034MA,S2401_C05_034MA,S2401_C05_034EA,S2401_C01_035EA,S2401_C01_035MA,S2401_C02_035MA,S2401_C02_035EA,S2401_C03_035MA,S2401_C03_035EA,S2401_C04_035EA,S2401_C04_035MA,S2401_C05_035EA,S2401_C05_035MA,S2401_C01_036MA,S2401_C01_036EA,S2401_C02_036MA,S2401_C02_036EA,S2401_C03_036MA,S2401_C03_036EA,S2401_C04_036MA,S2401_C04_036EA,S2401_C05_036MA,S2401_C05_036EA,zip code tabulation area
0,3564,989,344,208,136,80,10,30,40,225,68,19,129,9,340,209,131,769,231,63,44,19,202,161,112,786,428,358,263,0,148,115,757,279,319,159,8600000US43964,3564,321,1853,196,52.0,3.3,1711,201,48.0,3.3,989,186,304,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,43964
1,25800,9444,4072,2543,1529,1303,868,390,45,2830,618,239,1404,569,1239,769,470,4847,921,409,294,115,1514,1202,801,5727,2294,3433,1783,43,930,810,3999,1167,1521,1311,8600000US28216,25800,1067,12137,668,47.0,1.8,13663,736,53.0,1.8,9444,610,3542,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,28216
2,37198,21707,11310,6490,4820,4188,3309,666,213,3967,532,516,2008,911,2242,1904,338,3448,418,252,121,131,1567,329,882,8901,5500,3401,1327,0,762,565,1815,668,473,674,8600000US28277,37198,1089,20009,678,53.8,1.3,17189,746,46.2,1.3,21707,756,12079,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,28277
3,15182,6793,3792,2331,1461,1128,743,254,131,1267,341,215,508,203,606,392,214,1814,117,204,167,37,757,269,467,4007,2036,1971,813,12,503,298,1755,721,665,369,8600000US28278,15182,890,7655,546,50.4,2.2,7527,560,49.6,2.2,6793,449,3190,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,28278
4,11384,3855,1295,771,524,313,113,152,48,1577,196,110,1127,144,670,400,270,2554,375,354,139,215,1054,331,440,2989,1420,1569,690,0,332,358,1296,672,282,342,8600000US28303,11384,592,5632,465,49.5,2.6,5752,370,50.5,2.6,3855,347,1651,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,28303


In [563]:
# Selecting only occupations for which we have an essential indicator
essential_worker_list = pd.read_excel('acs_occupation_lookup.xlsx')
essential_worker_list.dropna(axis=0, subset=['essential','variable'], inplace=True)
essential_worker_list = essential_worker_list[essential_worker_list.essential==1]
essential_worker_list['variable'] = essential_worker_list.variable.apply(lambda x: x+'E')

In [564]:
# Confining the dataset to only those variables included in our variables list
df2 = pd.merge(df2.iloc[:,:36], df2.iloc[:,-1], left_index=True, right_index=True)

In [565]:
# Confining the dataset to just essential workers since we are mostly conserned about created pctEssential workers
df2 = pd.merge(df2.loc[:,list(essential_worker_list.variable)], df2.iloc[:,-1], left_index=True, right_index=True)

In [566]:
# Construct a crosswalk to change the column names
crosswalk2 = essential_worker_list[['variable', 'occupation']]
crosswalk2.set_index('variable', inplace=True)
new_col_names = []
for each in df2.columns[:-1]:
    new_col_names.append(crosswalk2.loc[each][0])

In [567]:
# Adding in zip code tabulation area since it's not in our excel sheet of var names
new_col_names.append('ZCTA')

In [568]:
# Rename columns
df2.columns = new_col_names

In [572]:
# Confine to Chicago zip codes only
df2_chi = df2[(df2['ZCTA']>='60601')&(df2['ZCTA']<='60701')|(df2['ZCTA']=='60707')|(df2['ZCTA']=='60827')]

In [573]:
# Create a new column for sum essential workers
df2_chi = df2_chi.astype(int)
df2_chi['sumEssential'] = df2_chi.iloc[:,:-1].sum(axis=1)
df2_chi.shape


(58, 19)

In [574]:
# Merge this column into larger census dataset from above to calculate percent essential workers
comm_chrs = pd.merge(census_df, df2_chi.iloc[:,-2:], on='ZCTA')

In [575]:
# Creating variable for percent of zip code population that is an essential worker
comm_chrs['pctEssential'] = comm_chrs.sumEssential/comm_chrs.totalPop

In [576]:
# Remove sumEssential since we no longer need it
comm_chrs.drop(['sumEssential'], axis=1, inplace=True)

In [577]:
comm_chrs

,ZCTA,totalPop,medianAge,medianHHInc,mean_HHsize,pctHispanic,pctBlack,pctWhite,pctUndocumented,pctUndocumentedFBLA,pctUninsured,pctUnemployed,pctHealthWorkers,pctEssential
0,60601,14675.0,34.9,103243.0,1.655573,0.086814,0.055741,0.741670,0.239387,0.035434,0.029972,0.040341,0.083543,0.145963
1,60602,1244.0,30.6,157125.0,2.066445,0.065113,0.037781,0.681672,0.117363,0.047428,0.008842,0.000804,0.126206,0.245981
2,60603,1174.0,28.9,151765.0,2.081560,0.097956,0.032368,0.634583,0.160136,0.000000,0.069847,0.055366,0.080068,0.231687
3,60604,782.0,32.4,116250.0,1.632568,0.043478,0.056266,0.634271,0.162404,0.008951,0.003836,0.057545,0.067775,0.125320
4,60605,27519.0,33.9,111164.0,1.984782,0.058432,0.171772,0.612050,0.103601,0.013700,0.026408,0.025146,0.101821,0.252516
5,60606,3101.0,37.2,131477.0,1.488004,0.062883,0.023541,0.727507,0.108352,0.017736,0.023632,0.019026,0.069332,0.253467
6,60607,29591.0,30.1,93508.0,2.307650,0.083032,0.145112,0.581697,0.121152,0.009091,0.033990,0.046061,0.087155,0.239025
7,60608,79205.0,32.4,44043.0,3.090686,0.506862,0.176529,0.446083,0.154813,0.116483,0.151609,0.038179,0.054795,0.244366
8,60609,61495.0,31.7,35179.0,2.984760,0.534352,0.247142,0.447175,0.185869,0.162582,0.156531,0.077551,0.048492,0.274169
9,60610,39019.0,33.6,86803.0,1.676866,0.066660,0.149645,0.738179,0.063174,0.007740,0.038046,0.026449,0.075784,0.235244


In [578]:
import pickle
pickle_out = open('census_data_zcta.pickle', 'wb')
pickle.dump(comm_chrs, pickle_out)
pickle_out.close()